# Geographic Locations of Data Centers 

This notebook provides a visualization of the geographic locations of the data centers for various cloud providers to give a sense of the geographic coverage of cloud computing. The dataset can be found in the `fixtures` directory, and contains general locations for the following providers:

- Amazon Web Services
- Google Cloud Compute 
- Microsoft Azure 
- Digital Ocean 
- Linode 

In [1]:
%matplotlib notebook 

import pandas as pd 
import geopandas as geo 
import ipywidgets as widgets
import matplotlib.pyplot as plt 

from ipywidgets import interact
from geopandas.tools import sjoin
from shapely.geometry import Point 

In [2]:
def load_data(path="fixtures/locations.csv"):
    df = pd.read_csv(path)
    df['Available'] = df['Available'].apply(lambda v: v=='y')
    df['Coordinates'] = list(zip(df.Longitude, df.Latitude))
    df['Coordinates'] = df['Coordinates'].apply(Point)
    
    gdf = geo.GeoDataFrame(df, geometry='Coordinates')
    return gdf 

cities = load_data()
cities.head()

,Provider,Location,Name,Notes,Available,Latitude,Longitude,Coordinates
0,Microsoft Azure,"Abu Dhabi, UAE",UAE Central,NaN,False,24.488180,54.354950,POINT (54.35495 24.48818)
1,Digital Ocean,"Amsterdam, Netherlands",AMS2,NaN,True,52.370216,4.895168,POINT (4.895168 52.370216)
2,Digital Ocean,"Amsterdam, Netherlands",AMS3,NaN,True,52.370216,4.895168,POINT (4.895168 52.370216)
3,AWS,"Ashburn, Virginia",us-east-1,6 availability zones,True,39.042370,-77.487240,POINT (-77.48724 39.04237)
4,Google Cloud,"Ashburn, Virginia",us-east4,3 availability zones,True,39.042370,-77.487240,POINT (-77.48724 39.04237)


In [3]:
PROVIDERS = frozenset(cities['Provider'].unique())
COLORS = ["#3498db", "#2ecc71", "#e74c3c", "#f39c12", "#9b59b6"]
COLORMAP = dict(zip(PROVIDERS, COLORS))

In [4]:
@interact(providers=widgets.SelectMultiple(options=PROVIDERS, value=tuple(PROVIDERS), description="Providers"))
def plot_cloud_geography(providers=None):
    """
    Plot the cloud geography filtering by continent or provider if required. 
    """
    
    # Create the axes object to plot on 
    _, ax = plt.subplots(figsize=(9,6))
    ax.set_aspect('equal')
    
    # Load and filter the world map 
    world = geo.read_file(geo.datasets.get_path('naturalearth_lowres'))
    world = world[(world.pop_est>0) & (world.name!="Antarctica")]
    
    # Plot the world map 
    world.plot(ax=ax, color='#ecf0f1', edgecolor="#bdc3c7", linewidth=0.75)
    
    # Plot the providers
    if providers:
        providers = set(providers) & PROVIDERS
    else:
        providers = PROVIDERS
    
    for provider in providers:
        cities[cities['Provider']==provider].plot(
            ax=ax, marker='o', markersize=10, alpha=0.7, 
            label=provider, color=COLORMAP[provider]
        )

    ax.legend(loc='lower left', frameon=True)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.tight_layout()
    
    return ax 


# plot_cloud_geography(providers=['AWS'])

interactive(children=(SelectMultiple(description='Providers', index=(0, 1, 2, 3, 4), options=('Microsoft Azure ', 'AWS', 'Digital Ocean', 'Google Cloud', 'Linode'), value=('Microsoft Azure ', 'AWS', 'Digital Ocean', 'Google Cloud', 'Linode')), Output()), _dom_classes=('widget-interact',))